so... this is a personal tutor.
currently there are multiple reward designs for teh teacher and student  currently... spelling is important  
currently lots of room for improvement
... worst README ever...

In [1]:
###########################################
# Suppress matplotlib user warnings
# Necessary for newer version of matplotlib
import random
#import warnings
#warnings.filterwarnings("ignore", 
#                        category = UserWarning, module = "matplotlib")

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from textblob import TextBlob
import re

# Pretty display for notebooks
%matplotlib inline

# Generate Trivia

In [2]:
def generate_trivia_sentences(clean_data):
    sentences = clean_data.sentences
    del sentences[0]
    trivia_sentences = []
    for sentence in sentences:
        trivia = evaluate_sentence(sentence)
        if trivia:
            trivia_sentences.append(trivia)

    trivia_sentences = generate_trivia_choices(trivia_sentences)   
    
    return trivia_sentences

def generate_trivia_choices(trivia_sentences):
    trivia_sentences = pd.DataFrame(trivia_sentences)
    trivia_sentences['choices'] = ""
    choices = range(len(trivia_sentences))
    
    for c in choices:
        answer = [trivia_sentences.iat[c,0]]
        wrongs = random.sample(choices,4)
        for w in wrongs:
            answer.append(trivia_sentences.iat[w,0])
        random.shuffle(answer)
        trivia_sentences.at[w,'choices'] = answer
        
    trivia_sentences = trivia_sentences[trivia_sentences['choices'] != '']
    trivia_sentences = trivia_sentences.reset_index(drop=True)
    trivia_sentences['difficulty'] = range(1, len(trivia_sentences) + 1)
    return trivia_sentences

def evaluate_sentence(sentence):
    if sentence.tags[0][1] == 'RB' or len(sentence.words) < 6:
        # This sentence starts with an adverb or is less than five words long
        # and probably won't be a good fit
        return None

    tag_map = {word.lower(): tag for word, tag in sentence.tags}
    
    
    replace_nouns = []
    for word, tag in sentence.tags:
        # For now, only blank out non-proper nouns that don't appear in the article title
        if tag == 'NN':
            # Is it in a noun phrase? If so, blank out the last two words in that phrase
            for phrase in sentence.noun_phrases:
                if word in phrase:
                    # Blank out the last two words in this phrase
                    [replace_nouns.append(phrase_word) for phrase_word in phrase.split()[-2:]]
                    break

            # If we couldn't find the word in any phrases,
            # replace it on its own
            if len(replace_nouns) == 0:
                replace_nouns.append(word)
            break

    if len(replace_nouns) == 0:
        # Return none if we found no words to replace
        return None

    trivia = {
        'answer': ' '.join(replace_nouns)
    }

    # Blank out our replace words (only the first occurrence of the word in the sentence)
    replace_phrase = ' '.join(replace_nouns)
    blanks_phrase = ('__________ ' * len(replace_nouns)).strip()

    expression = re.compile(re.escape(replace_phrase), re.IGNORECASE)
    sentence = expression.sub(blanks_phrase, str(sentence), count=1)
    trivia['question'] = sentence.replace('\n', ' ')
    return trivia

def build_trivia(file_name):
    with open(file_name,'r') as f:
        raw_data = f.read()
    clean_data = TextBlob(raw_data)
    trivia_sentences = generate_trivia_sentences(clean_data)
    trivia = []
    for i in range(len(trivia_sentences)):  
        trivia.append([trivia_sentences.iat[i,1],
        trivia_sentences.iat[i,2],
        trivia_sentences.iat[i,0],
        trivia_sentences.iat[i,3]])
    return trivia

# Tutor

In [3]:
def getQuestion(difficulty, questions):
    question = []
    for q,a,d in questions:
        if d <= difficulty+2 or d >= difficulty-2:
            question.append([q,a,d])
    if len(question) != 0:
        question = random.choice(question)
    else:
        question = questions[0]
    return question

"""
teacher should be moving the bar up and down 
"""
def teacher(accurate, difficult):
    if (accurate == True and difficult == True):
        change = 1    
    elif (accurate == True and difficult == False):
        change = 0   
    elif (accurate == False and difficult == True):
        change = 0    
    else:
        change = -1 

    #print( "Accurate: %r" % accurate, "Difficult: %r" % difficult)
    #print( "Teacher changed difficulty by {}".format(change) )

    return change

def getRewardP1(accurate, difficult, sPoints, tPoints):
    if accurate:
        sPoints += 1
        tPoints += 1
    else:
        pass
    #print(sPoints,tPoints)
    return(sPoints,tPoints)

    
def getRewardP2(accurate, difficult, sPoints, tPoints):
    if accurate:
        sPoints += 1
        tPoints -= 1
    else:
        pass
    #print(sPoints,tPoints)
    return(sPoints,tPoints)
    
def getRewardP3(accurate, difficult, sPoints, tPoints):
    if accurate:
        sPoints += 1
    else:
        tPoints += 1
    #print(sPoints,tPoints)
    return(sPoints,tPoints)
    
def getRewardP4(accurate, difficult, sPoints, tPoints):
    if accurate and difficult:
        sPoints += 1
        tPoints += 1
    elif accurate and difficult != True:
        sPoints += 1
    elif accurate != True and difficult != True:
        tPoints += 1
    else:
        pass
    #print(sPoints,tPoints)
    return(sPoints,tPoints)

# Student

In [4]:
# https://stackoverflow.com/questions/423379/using-global-variables-in-a-function-other-than-the-one-that-created-them

difficult = True
accurate = True
difficulty = 0
sPoints = 0
tPoints = 0
output = []

def Huh(trivia, getReward=getRewardP1):
    """
    set globals
    """
    global accurate
    global difficult
    global difficulty
    global sPoints
    global tPoints
    global output
    trial = 0
    for each_question, each_answer, each_correct_answer, qDifficulty in trivia:
        trial += 1
        """
        play trivia
        """
        print(each_question + '\n' + ' \n'.join(each_answer))
        get_answer = input()
        
        """
        update difficult and accurate
        """
        if qDifficulty >= difficulty:
            difficult = True
        else:
            difficult = False
            
        if get_answer in each_correct_answer:
            accurate = True
            print(get_answer,'is correct!\n')
        else:
            accurate = False
            print(get_answer, 'is not the answer I had in mind!\n')
        
        """
        update change
        update difficulty bar
        update ponts
        update list of results for plotting
        """
        change = teacher(accurate, difficult)
        difficulty += change
        sPoints, tPoints = getReward(accurate, difficult, sPoints, tPoints)
        output.append([trial,difficulty,sPoints,tPoints])
        output = pd.DataFrame(output, columns = ['trial', 'difficulty', 'sPoints', 'tPoints'])
        print("trial:", trial, "\n",
              "difficulty:", difficulty, "\n",
              "change:", change, "\n",
              "sPoints:",sPoints, "\n",
              "tPoints:", tPoints, "\n",
              "accurate:", accurate,  "\n",
              "difficult:", difficult)


# What makes you say Huh?

In [ ]:
file_name = 'katana.txt'
trivia = build_trivia(file_name);


In [ ]:
Huh(trivia)

The first __________ of "katana" (gatana) as a word to describe a long sword that was different from a tachi occurs as early as the Kamakura Period (1185–1333).
tachi 
period 
's signature 
use 
century
